<a href="https://colab.research.google.com/github/ThomasGVoss/LearningFactory/blob/main/Lab_Discrete_Event_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We will be using SimPy to model the learning factory

Documentation can be found:  [here](https://simpy.readthedocs.io/en/latest/)

In [ ]:
!pip install simpy

In [ ]:
import simpy
import random

In [ ]:
class Flow_Shop:
    def __init__(self, env):
        self.raw = simpy.Container(env, capacity = 1000, init = 20)
        self.s1 = simpy.Container(env, capacity = 1000, init = 0)
        self.dispatch = simpy.Container(env ,capacity = 1000, init = 0)

In [ ]:
# Define product types
class Product:
    def __init__(self, name):
        self.name = name

# List of products
products = [Product("A"), Product("B"), Product("C")]

# Setup time matrix (in minutes/hours as needed)
# setup_times[from_product][to_product]
setup_times = {
    "A": {"A": 0, "B": 2, "C": 3},
    "B": {"A": 2, "B": 0, "C": 4},
    "C": {"A": 3, "B": 4, "C": 0}
}

Stations can be used as 
https://simpy.readthedocs.io/en/latest/topical_guides/resources.html#shared-resources

In [ ]:
def station1(env, flow_shop, products, setup_times):
    last_product = None
    while True:
        # Randomly select next product
        product = random.choice(products)
        yield flow_shop.raw.get(1)
        # Setup time if switching products
        if last_product is not None:
            setup_time = setup_times[last_product.name][product.name]
            if setup_time > 0:
                print(f"Station 1 - Setup for {product.name} from {last_product.name} at time {env.now}")
                yield env.timeout(setup_time)
        body_time = random.randint(2, 4)
        print(f"Machine 1 - Start processing {product.name} at time {env.now}")
        yield env.timeout(body_time)
        yield flow_shop.s1.put(1)
        last_product = product

def station2(env, flow_shop, products, setup_times):
    last_product = None
    while True:
        product = random.choice(products)
        yield flow_shop.s1.get(1)
        # Setup time if switching products
        if last_product is not None:
            setup_time = setup_times[last_product.name][product.name]
            if setup_time > 0:
                print(f"Station 2 - Setup for {product.name} from {last_product.name} at time:{env.now}")
                yield env.timeout(setup_time)
        print(f"Machine 2 - Getting material for {product.name} from s1 which now has {flow_shop.s1.level:.0f} items left in store.")
        print(f"Machine 2 - Start processing {product.name} at time:{env.now}")
        body_time = random.randint(1, 5)
        yield env.timeout(body_time)
        yield flow_shop.dispatch.put(1)
        last_product = product

In [ ]:
#working hours
hours = 8
#business days
days = 5
#total working time (hours)
total_time = hours * days

env = simpy.Environment()
flow_shop = Flow_Shop(env)

station1_proc = env.process(station1(env, flow_shop, products, setup_times))
station2_proc = env.process(station2(env, flow_shop, products, setup_times))

env.run(until = total_time)
print(f'Dispatch has %d bodies ready to go!' % flow_shop.dispatch.level)